In [1]:
import pandas as pd

# 데이터 파일 로드
ratings = pd.read_csv('Filtered_Ratings_NoZero.csv')

# User-ID를 기준으로 평점 개수를 세어서 200개 이상인 유저만 선택
user_counts = ratings['User-ID'].value_counts()
selected_users = user_counts[user_counts >= 200].index
filtered_ratings = ratings[ratings['User-ID'].isin(selected_users)]

# 새로운 파일로 저장
filtered_ratings.to_csv('Filtered_Ratings_200_or_more.csv', index=False)

print("새로운 파일이 저장되었습니다.")


새로운 파일이 저장되었습니다.


In [19]:
import pandas as pd

# 데이터 로드
data = pd.read_csv('Filtered_Ratings_200_or_more.csv')

# 랜덤 샘플링
random_users = data['User-ID'].unique()[:1000]
random_data = data[data['User-ID'].isin(random_users)]

# 결과를 CSV 파일로 저장
random_data.to_csv('Random_User_Rating.csv', index=False)


In [24]:
import pandas as pd

# 데이터 로드
data = pd.read_csv('Filtered_Ratings_200_or_more.csv')
books = pd.read_csv('Books.csv', encoding='ISO-8859-1')

# 사용자가 읽은 책
input_book = '037310507X'

# 해당 책을 읽은 유저들 찾기
users_who_read_input_book = data[data['ISBN'] == input_book]['User-ID']

# 이 유저들이 평점을 7점 이상 준 도서 찾기
high_rated_books = data[(data['User-ID'].isin(users_who_read_input_book)) & (data['Book-Rating'] >= 7)]

# 빈도수가 가장 높은 도서 5권 찾기
recommended_books = high_rated_books['ISBN'].value_counts().nlargest(5).index

# 추천된 도서의 제목 찾기
recommended_books_titles = books[books['ISBN'].isin(recommended_books)]['Book-Title']

print("도서 추천:")
print(recommended_books_titles)


도서 추천:
7170     Cowboy For Clementine (Home On The Ranch) (Har...
12557                                           Sweetwater
18030    Man From Blue River (Home On The Ranch) (Harle...
18057                  The Bandit's Lady (Harper Monogram)
18248    Texas Standoff  (Home On The Ranch) (Harlequin...
Name: Book-Title, dtype: object


C:\Users\asdf9\AppData\Local\Temp\ipykernel_10728\3378612624.py:5: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('Books.csv', encoding='ISO-8859-1')


In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori

# 데이터 로드
data = pd.read_csv('Random_User_Rating.csv')
books = pd.read_csv('Books.csv', encoding='ISO-8859-1')

# 특정 유저 선택 (임의로 User-ID를 설정)
selected_user = data['User-ID'].sample(1).values[0]

# 선택한 유저가 읽은 도서 데이터 중 평점이 7점 이상인 도서 선택
user_high_rated_books = data[(data['User-ID'] == selected_user) & (data['Book-Rating'] >= 7)]

# 선택한 유저가 읽은 도서들에 평점을 7점 이상 준 다른 유저들의 도서 데이터
other_users_data = data[(data['ISBN'].isin(user_high_rated_books['ISBN'])) & (data['User-ID'] != selected_user) & (data['Book-Rating'] >= 7)]

# 데이터 전처리 - 이진 데이터로 변환
basket = (other_users_data.groupby(['User-ID', 'ISBN'])['Book-Rating']
          .sum().unstack().reset_index().fillna(0)
          .set_index('User-ID').applymap(lambda x: 1 if x > 0 else 0))

# Apriori 알고리즘을 사용하여 연관 규칙 생성
frequent_itemsets = apriori(basket, min_support=0.01, use_colnames=True)

# 연관된 도서들 중 빈도수가 가장 높은 도서 3권 중 1권추천
recommended_books = frequent_itemsets.nlargest(3, 'support')['itemsets'].values[0]

# 추천된 도서의 제목 찾기
recommended_books_titles = books[books['ISBN'].isin(recommended_books)]['Book-Title']

print(f"User {selected_user}에게 도서 추천:")
print(recommended_books_titles)


C:\Users\asdf9\AppData\Local\Temp\ipykernel_17276\1153607061.py:6: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('Books.csv', encoding='ISO-8859-1')
C:\Users\asdf9\AppData\Local\Temp\ipykernel_17276\1153607061.py:20: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  .set_index('User-ID').applymap(lambda x: 1 if x > 0 else 0))
C:\Users\asdf9\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


User 11676에게 도서 추천:
2143    Harry Potter and the Sorcerer's Stone (Harry P...
Name: Book-Title, dtype: object


In [15]:
import pandas as pd
from mlxtend.frequent_patterns import apriori

# 데이터 로드
data = pd.read_csv('Random_User_Rating.csv')
books = pd.read_csv('Books.csv', encoding='ISO-8859-1')

# 특정 유저 선택 (임의로 User-ID를 설정)
selected_user = data['User-ID'].sample(1).values[0]

# 선택한 유저가 읽은 도서 데이터 중 평점이 7점 이상인 도서 선택
user_high_rated_books = data[(data['User-ID'] == selected_user) & (data['Book-Rating'] >= 7)]

# 선택한 유저가 읽은 도서들에 평점을 7점 이상 준 다른 유저들의 도서 데이터
other_users_data = data[(data['ISBN'].isin(user_high_rated_books['ISBN'])) & (data['User-ID'] != selected_user) & (data['Book-Rating'] >= 7)]

# 데이터 전처리 - 이진 데이터로 변환
basket = (other_users_data.groupby(['User-ID', 'ISBN'])['Book-Rating']
          .sum().unstack().reset_index().fillna(0)
          .set_index('User-ID').applymap(lambda x: 1 if x > 0 else 0))

# Apriori 알고리즘을 사용하여 연관 규칙 생성
frequent_itemsets = apriori(basket, min_support=0.01, use_colnames=True)

# 연관된 도서들 중 빈도수가 가장 높은 도서 3권 중 1권추천
recommended_books = frequent_itemsets.nlargest(3, 'support')['itemsets'].values[0]

# 추천된 도서의 제목 찾기
recommended_books_titles = books[books['ISBN'].isin(recommended_books)]['Book-Title']

print(f"User {selected_user}에게 도서 추천:")
print(recommended_books_titles)


C:\Users\asdf9\AppData\Local\Temp\ipykernel_19496\1753462460.py:6: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('Books.csv', encoding='ISO-8859-1')
C:\Users\asdf9\AppData\Local\Temp\ipykernel_19496\1753462460.py:20: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  .set_index('User-ID').applymap(lambda x: 1 if x > 0 else 0))
C:\Users\asdf9\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


User 38273에게 도서 추천:
6196    Full House (Janet Evanovich's Full Series)
Name: Book-Title, dtype: object


In [18]:
import pandas as pd
from mlxtend.frequent_patterns import apriori

# 데이터 로드
data = pd.read_csv('Random_User_Rating.csv')
books = pd.read_csv('Books.csv', encoding='ISO-8859-1')

# 특정 유저 선택 (임의로 User-ID를 설정)
selected_user = data['User-ID'].sample(1).values[0]

# 선택한 유저가 읽은 도서 데이터 중 평점이 7점 이상인 도서 선택
user_high_rated_books = data[(data['User-ID'] == selected_user) & (data['Book-Rating'] >= 7)]

# 선택한 유저가 읽은 도서들에 평점을 7점 이상 준 다른 유저들의 도서 데이터
other_users_data = data[(data['ISBN'].isin(user_high_rated_books['ISBN'])) & (data['User-ID'] != selected_user) & (data['Book-Rating'] >= 7)]

# 데이터 전처리 - 이진 데이터로 변환
basket = (other_users_data.groupby(['User-ID', 'ISBN'])['Book-Rating']
          .sum().unstack().reset_index().fillna(0)
          .set_index('User-ID').applymap(lambda x: 1 if x > 0 else 0))

# Apriori 알고리즘을 사용하여 연관 규칙 생성
frequent_itemsets = apriori(basket, min_support=0.01, use_colnames=True)

# 연관된 도서들 중 빈도수가 가장 높은 도서 1권 추천 
recommended_books = frequent_itemsets.nlargest(1, 'support')['itemsets'].values[0]

# 추천된 도서의 제목 찾기
recommended_books_titles = books[books['ISBN'].isin(recommended_books)]['Book-Title']

print(f"User {selected_user}에게 도서 추천:")
print(recommended_books_titles)


C:\Users\asdf9\AppData\Local\Temp\ipykernel_19496\941276213.py:6: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('Books.csv', encoding='ISO-8859-1')
C:\Users\asdf9\AppData\Local\Temp\ipykernel_19496\941276213.py:20: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  .set_index('User-ID').applymap(lambda x: 1 if x > 0 else 0))
C:\Users\asdf9\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


User 105517에게 도서 추천:
408    The Lovely Bones: A Novel
Name: Book-Title, dtype: object


In [20]:
import pandas as pd
from mlxtend.frequent_patterns import apriori

# 데이터 로드
data = pd.read_csv('Random_User_Rating.csv')
books = pd.read_csv('Books.csv', encoding='ISO-8859-1')

# 특정 유저 선택 (임의로 User-ID를 설정)
selected_user = data['User-ID'].sample(1).values[0]

# 선택한 유저가 읽은 도서 데이터 중 평점이 7점 이상인 도서 선택
user_high_rated_books = data[(data['User-ID'] == selected_user) & (data['Book-Rating'] >= 7)]

# 선택한 유저가 읽은 도서들에 평점을 7점 이상 준 다른 유저들의 도서 데이터
other_users_data = data[(data['ISBN'].isin(user_high_rated_books['ISBN'])) & (data['User-ID'] != selected_user) & (data['Book-Rating'] >= 7)]

# 데이터 전처리 - 이진 데이터로 변환
basket = (other_users_data.groupby(['User-ID', 'ISBN'])['Book-Rating']
          .sum().unstack().reset_index().fillna(0)
          .set_index('User-ID').applymap(lambda x: 1 if x > 0 else 0))

# Apriori 알고리즘을 사용하여 연관 규칙 생성
frequent_itemsets = apriori(basket, min_support=0.01, use_colnames=True)

# 연관된 도서들 중 빈도수가 가장 높은 도서 1권 추천 
recommended_books = frequent_itemsets.nlargest(1, 'support')['itemsets'].values[0]

# 추천된 도서의 제목 찾기
recommended_books_titles = books[books['ISBN'].isin(recommended_books)]['Book-Title']

print(f"User {selected_user}에게 도서 추천:")
print(recommended_books_titles)


C:\Users\asdf9\AppData\Local\Temp\ipykernel_19496\941276213.py:6: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('Books.csv', encoding='ISO-8859-1')
C:\Users\asdf9\AppData\Local\Temp\ipykernel_19496\941276213.py:20: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  .set_index('User-ID').applymap(lambda x: 1 if x > 0 else 0))
C:\Users\asdf9\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


User 63714에게 도서 추천:
4320    The Darwin Awards II: Unnatural Selection
Name: Book-Title, dtype: object


In [21]:
import pandas as pd

# 데이터 로드
ratings_data = pd.read_csv('Filtered_Ratings_NoZero.csv')

# 유저별 평점 개수 계산
user_ratings_count = ratings_data['User-ID'].value_counts()

# 평점 개수가 100개 이상 200개 이하인 유저들의 ID 추출
selected_users = user_ratings_count[(user_ratings_count >= 100) & (user_ratings_count <= 200)].index

# 해당 유저들의 정보만 필터링
filtered_ratings_data = ratings_data[ratings_data['User-ID'].isin(selected_users)]

# 결과를 새로운 CSV 파일로 저장
filtered_ratings_data.to_csv('Filtered_Ratings_SelectedUsers.csv', index=False)

print("새로운 파일이 생성되었습니다: Filtered_Ratings_SelectedUsers.csv")


새로운 파일이 생성되었습니다: Filtered_Ratings_SelectedUsers.csv


In [22]:
import pandas as pd
from mlxtend.frequent_patterns import apriori

# 데이터 로드
data = pd.read_csv('Filtered_Ratings_SelectedUsers.csv')
books = pd.read_csv('Books.csv', encoding='ISO-8859-1')

# 특정 유저 선택 (임의로 User-ID를 설정)
selected_user = data['User-ID'].sample(1).values[0]

# 선택한 유저가 읽은 도서 데이터 중 평점이 7점 이상인 도서 선택
user_high_rated_books = data[(data['User-ID'] == selected_user) & (data['Book-Rating'] >= 7)]

# 선택한 유저가 읽은 도서들에 평점을 7점 이상 준 다른 유저들의 도서 데이터
other_users_data = data[(data['ISBN'].isin(user_high_rated_books['ISBN'])) & (data['User-ID'] != selected_user) & (data['Book-Rating'] >= 7)]

# 데이터 전처리 - 이진 데이터로 변환
basket = (other_users_data.groupby(['User-ID', 'ISBN'])['Book-Rating']
          .sum().unstack().reset_index().fillna(0)
          .set_index('User-ID').applymap(lambda x: 1 if x > 0 else 0))

# Apriori 알고리즘을 사용하여 연관 규칙 생성
frequent_itemsets = apriori(basket, min_support=0.01, use_colnames=True)

# 연관된 도서들 중 빈도수가 가장 높은 도서 1권 추천 
recommended_books = frequent_itemsets.nlargest(1, 'support')['itemsets'].values[0]

# 추천된 도서의 제목 찾기
recommended_books_titles = books[books['ISBN'].isin(recommended_books)]['Book-Title']

print(f"User {selected_user}에게 도서 추천:")
print(recommended_books_titles)


User 201290에게 도서 추천:
5431    Harry Potter and the Goblet of Fire (Book 4)
Name: Book-Title, dtype: object


C:\Users\asdf9\AppData\Local\Temp\ipykernel_19496\2063814663.py:6: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('Books.csv', encoding='ISO-8859-1')
C:\Users\asdf9\AppData\Local\Temp\ipykernel_19496\2063814663.py:20: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  .set_index('User-ID').applymap(lambda x: 1 if x > 0 else 0))
C:\Users\asdf9\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [23]:
import pandas as pd
from mlxtend.frequent_patterns import apriori

# 데이터 로드
data = pd.read_csv('Filtered_Ratings_SelectedUsers.csv')
books = pd.read_csv('Books.csv', encoding='ISO-8859-1')

# 특정 유저 선택 (임의로 User-ID를 설정)
selected_user = data['User-ID'].sample(1).values[0]

# 선택한 유저가 읽은 도서 데이터 중 평점이 7점 이상인 도서 선택
user_high_rated_books = data[(data['User-ID'] == selected_user) & (data['Book-Rating'] >= 7)]

# 선택한 유저가 읽은 도서들에 평점을 7점 이상 준 다른 유저들의 도서 데이터
other_users_data = data[(data['ISBN'].isin(user_high_rated_books['ISBN'])) & (data['User-ID'] != selected_user) & (data['Book-Rating'] >= 7)]

# 데이터 전처리 - 이진 데이터로 변환
basket = (other_users_data.groupby(['User-ID', 'ISBN'])['Book-Rating']
          .sum().unstack().reset_index().fillna(0)
          .set_index('User-ID').applymap(lambda x: 1 if x > 0 else 0))

# Apriori 알고리즘을 사용하여 연관 규칙 생성
frequent_itemsets = apriori(basket, min_support=0.01, use_colnames=True)

# 연관된 도서들 중 빈도수가 가장 높은 도서 1권 추천 
recommended_books = frequent_itemsets.nlargest(1, 'support')['itemsets'].values[0]

# 추천된 도서의 제목 찾기
recommended_books_titles = books[books['ISBN'].isin(recommended_books)]['Book-Title']

print(f"User {selected_user}에게 도서 추천:")
print(recommended_books_titles)


User 244627에게 도서 추천:
18    The Testament
Name: Book-Title, dtype: object


C:\Users\asdf9\AppData\Local\Temp\ipykernel_19496\2063814663.py:6: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('Books.csv', encoding='ISO-8859-1')
C:\Users\asdf9\AppData\Local\Temp\ipykernel_19496\2063814663.py:20: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  .set_index('User-ID').applymap(lambda x: 1 if x > 0 else 0))
C:\Users\asdf9\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [29]:
import pandas as pd
from mlxtend.frequent_patterns import apriori

# 데이터 로드
data = pd.read_csv('Filtered_Ratings_SelectedUsers.csv')
books = pd.read_csv('Books.csv', encoding='ISO-8859-1')

# 특정 유저 선택 (임의로 User-ID를 설정)
selected_user = data['User-ID'].sample(1).values[0]

# 선택한 유저가 읽은 도서 데이터 중 평점이 7점 이상인 도서 선택
user_high_rated_books = data[(data['User-ID'] == selected_user) & (data['Book-Rating'] >= 7)]

# 선택한 유저가 읽은 도서들에 평점을 7점 이상 준 다른 유저들의 도서 데이터
other_users_data = data[(data['ISBN'].isin(user_high_rated_books['ISBN'])) & (data['User-ID'] != selected_user) & (data['Book-Rating'] >= 7)]

# 데이터 전처리 - 이진 데이터로 변환
basket = (other_users_data.groupby(['User-ID', 'ISBN'])['Book-Rating']
          .sum().unstack().reset_index().fillna(0)
          .set_index('User-ID').applymap(lambda x: 1 if x > 0 else 0))

# Apriori 알고리즘을 사용하여 연관 규칙 생성
frequent_itemsets = apriori(basket, min_support=0.01, use_colnames=True)

# 연관된 도서들 중 빈도수가 가장 높은 도서 3권 추천 
recommended_books = frequent_itemsets.nlargest(3, 'support')['itemsets'].values

# 추천된 도서의 제목 찾기
recommended_books_titles = books[books['ISBN'].isin(recommended_books)]['Book-Title']

print(f"User {selected_user}에게 도서 추천:")
for book_set in recommended_books:
    recommended_books_titles = books[books['ISBN'].isin(book_set)]['Book-Title']
    print(recommended_books_titles)


User 247429에게 도서 추천:
3354    The Hobbit : The Enchanting Prelude to The Lor...
Name: Book-Title, dtype: object
1571    The Fellowship of the Ring (The Lord of the Ri...
Name: Book-Title, dtype: object
3730    Anne of Avonlea (Anne of Green Gables Novels (...
Name: Book-Title, dtype: object


C:\Users\asdf9\AppData\Local\Temp\ipykernel_19496\1278634032.py:6: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('Books.csv', encoding='ISO-8859-1')
C:\Users\asdf9\AppData\Local\Temp\ipykernel_19496\1278634032.py:20: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  .set_index('User-ID').applymap(lambda x: 1 if x > 0 else 0))
C:\Users\asdf9\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [37]:
import pandas as pd
from mlxtend.frequent_patterns import apriori

# 데이터 로드
data = pd.read_csv('Random_User_Rating.csv')
books = pd.read_csv('Books.csv', encoding='ISO-8859-1')

# 특정 유저 선택 (임의로 User-ID를 설정)
selected_user = data['User-ID'].sample(1).values[0]

# 선택한 유저가 읽은 도서 데이터 중 평점이 7점 이상인 도서 선택
user_high_rated_books = data[(data['User-ID'] == selected_user) & (data['Book-Rating'] >= 7)]

# 선택한 유저가 읽은 도서들에 평점을 7점 이상 준 다른 유저들의 도서 데이터
other_users_data = data[(data['ISBN'].isin(user_high_rated_books['ISBN'])) & (data['User-ID'] != selected_user) & (data['Book-Rating'] >= 7)]

# 데이터 전처리 - 이진 데이터로 변환
basket = (other_users_data.groupby(['User-ID', 'ISBN'])['Book-Rating']
          .sum().unstack().reset_index().fillna(0)
          .set_index('User-ID').applymap(lambda x: 1 if x > 0 else 0))

# Apriori 알고리즘을 사용하여 연관 규칙 생성
frequent_itemsets = apriori(basket, min_support=0.01, use_colnames=True)

# 연관된 도서들 중 빈도수가 가장 높은 도서 3권 추천 
recommended_books = frequent_itemsets.nlargest(3, 'support')['itemsets'].values

# 추천된 도서의 제목 찾기
recommended_books_titles = books[books['ISBN'].isin(recommended_books)]['Book-Title']

print(f"User {selected_user}에게 도서 추천:")
for book_set in recommended_books:
    recommended_books_titles = books[books['ISBN'].isin(book_set)]['Book-Title']
    print(recommended_books_titles)


C:\Users\asdf9\AppData\Local\Temp\ipykernel_19496\2035278999.py:6: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('Books.csv', encoding='ISO-8859-1')
C:\Users\asdf9\AppData\Local\Temp\ipykernel_19496\2035278999.py:20: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  .set_index('User-ID').applymap(lambda x: 1 if x > 0 else 0))
C:\Users\asdf9\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


User 35859에게 도서 추천:
2143    Harry Potter and the Sorcerer's Stone (Harry P...
Name: Book-Title, dtype: object
522    The Red Tent (Bestselling Backlist)
Name: Book-Title, dtype: object
1863    1st to Die: A Novel
Name: Book-Title, dtype: object
